# Install the charting library & tables

In [ ]:
!pip install plotly
!pip install tabulate

## Login 

In [ ]:
import requests
import os
import pprint
from dotenv import load_dotenv

# Define the API endpoint and payload
base_url = "https://api-nwfp.rothamsted.ac.uk/api"
endpoint = "auth/v1/login/"
url = f"{base_url}/{endpoint}"

load_dotenv()

payload = {
    "email": os.getenv("API_EMAIL"),
    "password": os.getenv("API_PASSWORD")
}
# Make the POST request
response = requests.post(url, data=payload)

# Check the response
if response.status_code == 200:
    response_data = response.json()
    print("Login successful!")
    token = response_data.get("access")
    headers = {
        "Authorization":f"Bearer {token}",
        "Cache-Control": "no-cache"
    }
    key = token
    print(key)
else:
    print("Failed to login. Status code:", response.status_code)
    print("Response:", response.text)

## Getting help about animals

The NWFP APIs can provide help information, for example if you want to know the available animalCategoryName values we can query using the *Help* parameter. This will tell us what the avalable animalCategoryName values are.

In [35]:
import requests
import pandas as pd
from tabulate import tabulate

# Define the API endpoint and parameters
endpoint = "animals/v1/getBasicAnimalData/"
api_url = f"{base_url}/{endpoint}"
params = {
    "Help": "animalCategoryName"
}

response = requests.get(api_url, headers=headers, params=params)

if response.status_code == 200:    
    data = response.json()
    df = pd.DataFrame(data)
    # Reformat the output in a nice table
    print(tabulate(df, headers='keys', tablefmt='grid'))
else:
    print("Failed. Status code:", response.status_code)
    print("Response:", response.text)

Failed. Status code: 400
Response: {"error":"Missing required parameters","missing_params":["animalCategoryName","startGrazingYear","endGrazingYear"]}


Now we know one of the animal categories is *Lamb*, we can query demographic data for Lambs using the *animals/v1/getBasicAnimalData/* end point. This example simply prints the returned JSON 

In [36]:
params = {
    "animalCategoryName":"Lamb",
    "startGrazingYear":2012,
    "endGrazingYear":2012,
    "farmlet":"Red"
}

response = requests.get(api_url, headers=headers, params=params)

if response.status_code == 200:    
    data = response.json()
    pprint.pp(data)
else:
    print("Failed. Status code:", response.status_code)
    print("Response:", response.text)

{'category': 'Lamb',
 'farmlet': 'Varies',
 'grazing years': ['2012', '2012'],
 'results': [{'id': 474,
              'officialtag': 'UK 0 362735 01392',
              'managementtag': '1392',
              'breed': 'LLEYN',
              'dateofbirth': '2012-03-22T00:00:00Z',
              'grazingyear': 2012,
              'endgrazingyear': 2012,
              'gender': 'F',
              'farmletid': 3,
              'sireid': None,
              'birthdamid': 925,
              'rearingdamid': 925,
              'birthlittersize': 2,
              'rearinglittersize': 2,
              'animalid': 1093,
              'animalcategoryid': 1,
              'breedinganimal': False,
              'farmletName': 'Blue'},
             {'id': 475,
              'officialtag': 'UK 0 362735 01393',
              'managementtag': '1393',
              'breed': 'LLEYN',
              'dateofbirth': '2012-03-22T00:00:00Z',
              'grazingyear': 2012,
              'endgrazingyear': 2012,


There is a lot of data! We can use tabulate to present a table of selected columns

In [37]:
df_results = pd.DataFrame(data.get('results', []))

columns = ["animalid", "officialtag", "managementtag", "breed", "gender", "dateofbirth"]

df_animals = df_results[columns]

print(tabulate(df_animals, headers='keys', tablefmt='grid', showindex=False))

+------------+-------------------+-----------------+---------+----------+----------------------+
|   animalid | officialtag       |   managementtag | breed   | gender   | dateofbirth          |
+============+===================+=================+=========+==========+======================+
|       1093 | UK 0 362735 01392 |            1392 | LLEYN   | F        | 2012-03-22T00:00:00Z |
+------------+-------------------+-----------------+---------+----------+----------------------+
|       1094 | UK 0 362735 01393 |            1393 | LLEYN   | F        | 2012-03-22T00:00:00Z |
+------------+-------------------+-----------------+---------+----------+----------------------+
|       1095 | UK 0 362735 01405 |            1405 | TEX     | F        | 2012-03-22T00:00:00Z |
+------------+-------------------+-----------------+---------+----------+----------------------+
|       1096 | UK 0 362735 01406 |            1406 | TEX     | F        | 2012-03-22T00:00:00Z |
+------------+----------------

## Querying livestock performance data
Animal data is found across several different endpoints, however, animal data uses a common ID, *animalid* and we can use that to join results from different queries. For example we can query the livestock performance data and join this to the data we've already queried.

In [46]:
endpoint = "animals/v1/getLivestockPerformanceSales/"
api_url = f"{base_url}/{endpoint}"

# extract the animalIDs from the previous query
animalIds = ",".join(map(str, df_animals["animalid"]))

params = {
    "animalId": animalIds,
    "yearSold": 2012
}

response = requests.get(api_url, headers=headers, params=params)

if response.status_code == 200:    
    data = response.json()
    df_sales = pd.DataFrame(data)
    
    df_merged = 
    
    # Reformat the output in a nice table
    print(tabulate(df, headers='keys', tablefmt='grid'))
else:
    print("Failed. Status code:", response.status_code)
    print("Response:", response.text)



1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128,1129,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179,1180,1181,1182,1183,1184,1185,1186,1187,1188,1189,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1215,1216,1217,1218,1219,1220,1221,1222,1223,1224,1225,1226,1227,1228,1229,1230,1231,1232,1233,1234,1235,1236,1237,1238,1239,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280,1281,1282,1283,1284,1285,1286,1287,1288,1289,1290,1291,1292,